In [ ]:

import os
model_save_path = "/home/sys1/Dong/A_Stage_Two/ABSA-QUAD/data/clean/uausal/desiontree"  #名字
train_data_path = "/home/sys1/Dong/A_Stage_Two/ABSA-QUAD/data/clean/uausal/train1.txt"
validate_data_path = "/home/sys1/Dong/A_Stage_Two/ABSA-QUAD/data/clean/uausal/dev1.txt"
test_data_path = "/home/sys1/Dong/A_Stage_Two/ABSA-QUAD/data/clean/uausal/test1.txt"
test_data_predict_out_path = "/home/sys1/Dong/A_Stage_Two/ABSA-QUAD/data/clean/uausal/" 

In [ ]:
import pandas as pd
import jieba
import json

label2id = {'happy': 0, 'angry': 1, 'sad': 2, 'fear': 3, 'surprise': 4, 'neutral': 5}
def load_data(filename):
    D = []
    f = json.load(open(filename))    
    for l in f:
        label, content = label2id[l['label']],l['content']
        D.append((label, content))
    return D

# 分词
def seg_words(contents):
    contents_segs = list()
    for content in contents:
        segs = jieba.lcut(content[1])
        # print(segs)
        contents_segs.append(" ".join(segs))

    return contents_segs
# content_train = load_data(train_data_path)
# s = seg_words(content_train)
# # print(s[1])

In [ ]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score,accuracy_score
from sklearn.tree import DecisionTreeClassifier
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] <%(processName)s> (%(threadName)s) %(message)s')
logger = logging.getLogger(__name__)


class TextClassifier():

    def __init__(self, vectorizer, classifier=MultinomialNB()):
        classifier = SVC(kernel="rbf")
        # classifier = KNeighborsClassifier(6, weights='uniform')
        # classifier = DecisionTreeClassifier()

        # classifier = SVC(kernel="linear")
        self.classifier = classifier
        self.vectorizer = vectorizer

    def features(self, x):
        return self.vectorizer.transform(x)

    def fit(self, x, y):

        self.classifier.fit(self.features(x), y)

    def predict(self, x):

        return self.classifier.predict(self.features(x))

    def score(self, x, y):
        return self.classifier.score(self.features(x), y)

    def get_f1_score(self, x, y):
        return f1_score(y, self.predict(x), average='macro')
    
    def get_acc_score(self, x, y):
        return accuracy_score(y, self.predict(x))

In [ ]:
#predict
import logging
import argparse
import joblib

logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] <%(processName)s> (%(threadName)s) %(message)s')
logger = logging.getLogger(__name__)

if __name__ == '__main__':

    model_name = "model_dict.pkl"

    # load data
    logger.info("start load data")
    content_test = load_data(test_data_path)

    # load model
    logger.info("start load model")
    classifier_dict = joblib.load(model_save_path + model_name)
    # print(classifier_dict)
    labels_test = []
    for l in content_test:
        labels_test.append(l[0])
    # seg words
    logger.info("start seg test data")
    content_test = seg_words(content_test)
    logger.info("complete seg test data")
    
    # model predict
    logger.info("start predict test data")


    # test_data_df= classifier_dict.predict(content_test)
    f1_score = classifier_dict.get_f1_score(content_test, labels_test)
    acc_score = classifier_dict.get_acc_score(content_test, labels_test)
    
    # test_data_df.to_csv(test_data_predict_out_path, encoding="utf_8_sig", index=False)
    logger.info("compete predict test data")
    logger.info("f1_scores: %s\n" % f1_score)
    logger.info("acc_score: %s" % acc_score)